In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/Shareddrives/CS263\ Project

Mounted at /content/drive
/content/drive/Shareddrives/CS263 Project


In [2]:
#Importing necessary libraries
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import keras
from sklearn.model_selection import train_test_split
from keras.metrics import Precision

In [3]:
#Input and read in spam dataset, check head of dataset
df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
#Create input vectors and match to correct column
X_input = df.v2
Y_input = df.v1

#Use label encoder to encode target labels (not input values)
labels = LabelEncoder()

Y = labels.fit_transform(Y_input)
Y = Y.reshape(-1,1)

In [5]:
#Split dataset into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_input, Y, random_state=0)
X_train = X_train.tolist()
X_test = X_test.tolist()

X_train = [text.lower() for text in X_train]
X_test = [text.lower() for text in X_test]

label2idx = {'ham':0,'spam':1}

In [6]:
#Tokenize columns
maxlen = 300
vocab_size = 10000
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)
train_seq = tokenizer.texts_to_sequences(X_train)
train_pad = keras.utils.pad_sequences(train_seq, maxlen=maxlen, truncating='post')
test_seq = tokenizer.texts_to_sequences(X_test)
test_pad = keras.utils.pad_sequences(test_seq, maxlen=maxlen, truncating='post')

In [7]:
#Function to add all required layers in Bi-directional LSTM model
def LSTM_Model():
    model = keras.Sequential([
        keras.layers.Embedding(vocab_size, 128, input_length=maxlen),
        keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(32)),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid'),
        keras.layers.Dropout(0.5)

    ])
    return model

In [15]:
#Summarize and then compile model
model = LSTM_Model()
model.summary()
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy', 'Precision', 'Recall'])
#Fit model on trainining data
model.fit(train_pad, y_train, epochs=10, validation_split=0.1)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 300, 128)          1280000   
                                                                 
 bidirectional_8 (Bidirectio  (None, 300, 128)         98816     
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 256)               16640     
                                                                 
 dense_9 (Dense)             (None, 1)                 257       
                                                                 
 dropout_4 (Dropout)         (None, 1)                

In [16]:
#check accuracy and precision on test data
# tok = Tokenizer(num_words=300)
# test_sequences = tok.texts_to_sequences(X_test)
# test_sequences_matrix = keras.utils.pad_sequences(test_sequences,maxlen=300)
final_test_accuracy = model.evaluate(test_pad,y_test)

44/44 [==============================] - 7s 152ms/step - loss: 0.1220 - accuracy: 0.9749 - precision: 0.9355 - recall: 0.8832
